In [1]:
import personal_token
from tinkoff.invest import Client, InstrumentStatus, SharesResponse, InstrumentIdType
import pandas as pd
import openpyxl

In [2]:
TOKEN = personal_token.token

In [21]:
def get_favourite_bonds():
    favourites_table = pd.DataFrame(columns=['figi', 'ticker', 'class_code', 'isin', 'instrument_type', 'otc_flag', 'api_trade_available_flag','instrument_kind'])
    favourites_table_list = [None] * 8

    with Client(TOKEN) as client:
        my_favourites = client.instruments.get_favorites()
        
        for i in my_favourites.favorite_instruments:
            favourites_table_list[0] = (i.figi)
            favourites_table_list[1] = (i.ticker)
            favourites_table_list[2] = (i.class_code)
            favourites_table_list[3] = (i.isin)
            favourites_table_list[4] = (i.instrument_type)
            favourites_table_list[5] = (i.otc_flag)
            favourites_table_list[6] = (i.api_trade_available_flag)
            favourites_table_list[7] = (i.instrument_kind)

            if i.instrument_type == 'bond':
                favourites_table.loc[len(favourites_table)] = favourites_table_list
    # favourites_table
    return favourites_table
    # favourites_table.to_excel('favourites.xlsx', engine='openpyxl', index=False)

In [22]:
favourites_table = get_favourite_bonds()

In [26]:
favourites_table

,figi,ticker,class_code,isin,instrument_type,otc_flag,api_trade_available_flag,instrument_kind
0,TCS00A1074Q1,RU000A1074Q1,TQCB,RU000A1074Q1,bond,False,True,1
1,TCS00A106Z46,RU000A106Z46,TQCB,RU000A106Z46,bond,False,True,1
2,TCS00A106ZU6,RU000A106ZU6,TQCB,RU000A106ZU6,bond,False,True,1
3,TCS00A1070X5,RU000A1070X5,TQCB,RU000A1070X5,bond,False,True,1
4,BBG00JS9D851,XS1760786340,TQOD,XS1760786340,bond,False,True,1
...,...,...,...,...,...,...,...,...
198,BBG01HQGYGR5,RU000A106LL5,TQCB,RU000A106LL5,bond,False,True,1
199,TCS00A105YH8,RU000A105YH8,TQCB,RU000A105YH8,bond,False,True,1
200,TCS00A1042W6,RU000A1042W6,TQCB,RU000A1042W6,bond,False,True,1
201,BBG007Z5F748,SU29009RMFS6,TQOB,RU000A0JV4N8,bond,False,True,1


In [44]:
def get_bonds_price(figi, instrument_type):
    with (Client(TOKEN) as client):
        bond_price = client.market_data.get_last_prices(figi=[figi], instrument_id=[instrument_type])

        units = bond_price.last_prices[0].price.units
        nano = bond_price.last_prices[0].price.nano
        
        price = float(f"{units}.{str(nano)}")
        return price

In [109]:
favourites_table['price'] = float(0)
def write_price_in_df():
    for i in range(0, len(favourites_table)):
        figi = favourites_table.iloc[i]['figi']
        instrument_type = favourites_table.iloc[i]['instrument_type']
        
        price = get_bonds_price(figi, instrument_type)
        favourites_table.loc[i, 'price'] = price

In [110]:
write_price_in_df()

In [111]:
favourites_table

,figi,ticker,class_code,isin,instrument_type,otc_flag,api_trade_available_flag,instrument_kind,price
0,TCS00A1074Q1,RU000A1074Q1,TQCB,RU000A1074Q1,bond,False,True,1,101.69
1,TCS00A106Z46,RU000A106Z46,TQCB,RU000A106Z46,bond,False,True,1,100.27
2,TCS00A106ZU6,RU000A106ZU6,TQCB,RU000A106ZU6,bond,False,True,1,100.57
3,TCS00A1070X5,RU000A1070X5,TQCB,RU000A1070X5,bond,False,True,1,100.48
4,BBG00JS9D851,XS1760786340,TQOD,XS1760786340,bond,False,True,1,80.40
...,...,...,...,...,...,...,...,...,...
198,BBG01HQGYGR5,RU000A106LL5,TQCB,RU000A106LL5,bond,False,True,1,101.40
199,TCS00A105YH8,RU000A105YH8,TQCB,RU000A105YH8,bond,False,True,1,101.60
200,TCS00A1042W6,RU000A1042W6,TQCB,RU000A1042W6,bond,False,True,1,101.24
201,BBG007Z5F748,SU29009RMFS6,TQOB,RU000A0JV4N8,bond,False,True,1,103.33


In [47]:
if __name__ == "__main__":
    pass